In [37]:
import requests
from bs4 import BeautifulSoup

def print_grid_from_google_doc(url):
    """
    Retrieves and parses a Google Doc containing a 2D grid of Unicode characters
    and prints the grid, filling missing positions with spaces.

    Args:
        url: The URL of the published Google Doc.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
    except requests.exceptions.RequestException as e:
        print(f"Error fetching document: {e}")
        return

    soup = BeautifulSoup(response.content, 'html.parser')
    rows = soup.find_all('tr')

    grid = {}
    max_x = 0
    max_y = 0

    for row in rows:
        cells = row.find_all('td')
        if len(cells) == 3:
            # Skip header row by checking if x-coordinate is not a digit
            if not cells[0].text.strip().isdigit():
                continue
            try:
                x = int(cells[0].text.strip())
                y = int(cells[2].text.strip())
                char = cells[1].text.strip()
                grid[(x, y)] = char
                max_x = max(max_x, x)
                max_y = max(max_y, y)
            except ValueError:
                continue

    for y in range(max_y + 1):
        row_str = ""
        for x in range(max_x + 1):
            row_str += grid.get((x, y), " ")
        print(row_str)

if __name__ == "__main__":
    # url = "https://docs.google.com/document/d/e/2PACX-1vRMx5YQlZNa3ra8dYYxmv-QIQ3YJe8tbI3kqcuC7lQiZm-CSEznKfN_HYNSpoXcZIV3Y_O3YoUB1ecq/pub"
    url = "https://docs.google.com/document/d/e/2PACX-1vTER-wL5E8YC9pxDx43gk8eIds59GtUUk4nJo_ZWagbnrH0NFvMXIw6VWFLpf5tWTZIT9P9oLIoFJ6A/pub"
    print_grid_from_google_doc(url)


██░    ███░ ██████░    ███████░     ██░     ██░     ██████████░ ████████░    ████████░  
██░  ███░     ██░    ███░    ██░   ████░   ████░    ██░         ██░     ██░  ██░     ██░
██░███░       ██░   ███░           ██░██░ ██░██░    ██░         ██░      ██░ ██░     ██░
████░         ██░   ██░           ███░ ██░██░ ██░   ████████░   ██░      ██░ ████████░  
██░███░       ██░   ███░          ██░  █████░ ███░  ██░         ██░      ██░ ██░     ██░
██░  ███░     ██░    ███░    ██░ ███░   ███░   ██░  ██░         ██░     ██░  ██░     ██░
██░    ███░ ██████░    ███████░  ██░           ███░ ██████████░ ████████░    ████████░  


In [ ]:
import requests
from io import StringIO
import csv

def decode_message_from_google_doc(url):
    """
    Retrieves data from a Google Doc URL, parses it, and prints a grid of characters.

    Args:
        url: The URL of the published Google Doc.
    """

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        content = response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the document: {e}")
        return

    # Find the CSV data within the HTML (more robust than assuming a specific index)
    start_index = content.find('<!-- DATA START -->')
    end_index = content.find('<!-- DATA END -->')

    if start_index == -1 or end_index == -1:
      print("Could not find data section in the document.")
      return

    csv_data = content[start_index + len('<!-- DATA START -->'):end_index]

    # Use StringIO to treat the extracted string as a file
    try:
        reader = csv.reader(StringIO(csv_data), delimiter=',', quotechar='"')
        data = list(reader)
    except csv.Error as e:
        print(f"Error parsing CSV data: {e}")
        return


    # Determine grid dimensions
    max_x = 0
    max_y = 0
    for row in data:
        if row:  # Handle empty lines
            try:
                x = int(row[0])
                y = int(row[1])
                max_x = max(max_x, x)
                max_y = max(max_y, y)
            except (IndexError, ValueError) as e:
                print(f"Skipping malformed row: {row}. Error: {e}")
                continue



    # Create and populate the grid
    grid = [[' ' for _ in range(max_x + 1)] for _ in range(max_y + 1)]
    for row in data:
        if row:
            try:
                x = int(row[0])
                y = int(row[1])
                char = row[2]
                grid[y][x] = char
            except (IndexError, ValueError) as e:
                #Error is already handled in the previous loop
                pass

    # Print the grid
    for row in grid:
        print(''.join(row))

# Example Usage (replace with the actual URL)
if __name__ == "__main__":
    # Simplified example (using a local file for testing)
    # url = "simplified_example.txt" #Create a file with the google doc content for testing
    # with open(url, "w") as f:
    #     f.write("0,0,█\n0,1,█\n0,2,█\n1,0,▀\n1,1,▀\n1,2,\n")

    url = "https://docs.google.com/document/d/e/2PACX-1vRMx5YQlZNa3ra8dYYxmv-QIQ3YJe8tbI3kqcuC7lQiZm-CSEznKfN_HYNSpoXcZIV3Y_O3YoUB1ecq/pub"

    decode_message_from_google_doc(url)

OSError: [Errno 22] Invalid argument: 'https://docs.google.com/document/d/e/2PACX-1vRMx5YQlZNa3ra8dYYxmv-QIQ3YJe8tbI3kqcuC7lQiZm-CSEznKfN_HYNSpoXcZIV3Y_O3YoUB1ecq/pub'